In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re
import numpy as np
import time
from datetime import date,timedelta
import base64
import pytesseract
import cv2
from random import *

PATH = "C:\Program Files (x86)\chromedriver.exe"
tesseractPath = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
address = ""  # include public address

In [ ]:
# read image
def readScreenShot(screenshot):
    pytesseract.pytesseract.tesseract_cmd = tesseractPath
    img = cv2.imread(screenshot)
    text = pytesseract.image_to_string(img)
    return text
    
# Check to see if word was read by cv2
def checkForWord(textlist,screenshotText):
    textReturn = ""
    for word in textlist:
        if word in screenshotText:
            textReturn = word
    return textReturn

# crop  screen shot
def cropScreenShot(imput,output,x,y,h,w):
    image = cv2.imread(imput)
    crop_image = image[x:w, y:h]
    cv2.imwrite(output,crop_image)
#     cv2.imshow("Cropped", crop_image)
#     cv2.waitKey(0)


In [ ]:
options = Options()
options.headless = True
# driver = webdriver.Chrome(PATH,options = options)
driver = webdriver.Chrome(PATH)
time.sleep(5)

# Access site
driver.get("https://bitcoinker.com/")
time.sleep(15)

In [ ]:
worker = True
ct = 0
totalSucess = 0
while worker == True or totalSucess < 120:
    # Move page down to capture screenshot
    driver.execute_script("window.scrollTo(0, 800)")

    # load and decode captcha image
    knownText = ["Earn Cryptocurrency" "Stack Exchange","Click above","Click to earn"
    ,"Visit these","Earn Satoshi","Earn More Bits","BitCoin Mining","Block Chain","Make Money"]
    inputText = ""
    success = False

    while success == False:
        driver.find_element_by_xpath("//*[@id='adcopy-link-refresh']").click()
        time.sleep(10)
        driver.save_screenshot("screenshot_input.png")
        # Use when NOT headless mode
        cropScreenShot("screenshot_input.png","screenshot_output.png",278,100,500,298)
        # Use when in headless mode
        # cropScreenShot("screenshot_input.png","screenshot_output.png",295,100,430,320)
        textdata = readScreenShot("screenshot_output.png")
        
        inputText = checkForWord(knownText,textdata)
        if inputText == "":
            f = open("screenshotData.txt", "a")
            f.write("No Match Found: " + textdata)
            f.close()
            ct+=1
            
        if ct == 10:
            worker = False
            
        if inputText != "":
            success = True
            ct = 0
            print("Sucessful Input: " + inputText)

    if success == True:
        # Input Text
        inputAnswer = driver.find_element_by_xpath("//*[@id='adcopy_response']")
        driver.find_element_by_xpath("//*[@id='adcopy_response']").clear()
        inputAnswer.send_keys(inputText)

        # Input public address
        inputAddress = driver.find_element_by_xpath("//*[@id='post-2']/div[2]/form/div[5]/input")
        driver.find_element_by_xpath("//*[@id='post-2']/div[2]/form/div[5]/input").clear()
        inputAddress.send_keys(address)

        # Move page down to get to click button
        driver.execute_script("window.scrollTo(0, 1400)")

        # Claim reward
        driver.find_element_by_xpath("//*[@id='post-2']/div[2]/form/div[7]").click()
        
        totalSucess+=1

        # Wait between 5 - 12 minutes
        slpTime = randint(360, 480)
        ty_res = time.gmtime(slpTime)
        res = time.strftime("%H:%M:%S",ty_res)
        print("Wait Time: " + str(res))
        time.sleep(slpTime)